<a href="https://colab.research.google.com/github/akageyama/sgks-sfm/blob/master/Teppen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:

""" teppen.py
    English language training.
    Developed by Akira Kageyama on 2017.11.26. """

import sys, os, ast, random

#=====================================
class Word_or_sentense:
#=====================================

    word_or_sentense = ''

    def __init__(self,string):
        # e.g., Text('This is a test.')
        self.word_or_sentense = str(string).rstrip('\n')

    def __str__(self):
        return str(self.word_or_sentense)

    def input(self):
        string = str(input('Enter text >'))
        self.word_or_sentense = self.strip_rkey(string)
        print('am in Word_or_sentense. input. = ', self.word_or_sentense)


#=====================================
class Element:
#=====================================
    def __init__(self,ja,en,tag='new',ctr=0):
        self.japanese = Word_or_sentense(ja)
        self.english = Word_or_sentense(en)
        self.tag = Word_or_sentense(tag)
        self.ctr = int(ctr) 

    def get_str_japanese(self):
        return str(self.japanese)

    def get_str_english(self):
        return str(self.english)

    def __str__(self):
        res = str(self.japanese)
        res += ' § '
        res += str(self.english)
        res += ' § '
        res += str(self.tag)
        res += ' § '
        res += str(self.ctr)
        return res

    def increment_ctr(self):
        self.ctr += 1

    def set_tag(self,tag_): 
        tag = str(tag_)
        if tag=='f' or tag=='focus':
            self.tag = 'focus'
        elif tag=='s' or tag=='skip':
            self.tag = 'skip'



#=====================================
class Database:
#=====================================
    filename = ''
    database = []

    def __init__(self,filename_):
        self.filename = filename_
        if os.path.exists(self.filename):
            with open(self.filename,'r') as f:
                for l in f: 
                    items = l.split(' § ')
                    ja = Word_or_sentense(items[0])
                    en = Word_or_sentense(items[1])
                    tag = Word_or_sentense(items[2])
                    ctr = int(items[3])
                    self.database.append(Element(ja,en,tag,ctr))
        else:
            self.database_text = ''


    def add_new_element(self,ja,en):
        self.database.append(Element(ja,en))

    def get_element(self,i):
        return self.database[i]

    def __len__(self):
        return len(self.database)

    def save(self):
        with open(self.filename,'w') as f:
            for i in self.database:
                f.write(str(i)+'\n')

    def quiz_english_japanese(self,i):
        if self.get_element(i).tag == 'skip':
            return 'cont' # do nothing for this i and continue.
        print()
        message = '> ' + str(i+1)
        message += '-th ['
        message += str(self.database[i].ctr)
        message += ' counts]... Return | f(ocus) | s(kip) | q(uit)'
        print(message)
        print('.'*40 + '?')
        print(self.get_element(i).get_str_english())
        k = input('.'*40 + '?\n')
        if k=='q':
            return 'quit'
        elif k=='f':
            self.get_element(i).set_tag('focus')
            print(self.get_element(i).get_str_japanese())
            self.get_element(i).increment_ctr()
        elif k=='s':
            self.get_element(i).set_tag('skip')
            print(self.get_element(i).get_str_japanese())
            self.get_element(i).increment_ctr()
        elif k=='':
            print(self.get_element(i).get_str_japanese())
            self.get_element(i).increment_ctr()
        else:
            self.database[i].ctr = int(k)
        return 'cont'                

    def quiz_japanese_english(self,i):
        if self.get_element(i).tag == 'skip':
            return 'cont' # do nothing for this i and continue.
        print()
        message = '> ' + str(i+1)
        message += '-th ['
        message += str(self.database[i].ctr)
        message += ' counts]... Return | f(ocus) | s(kip) | q(uit)'
        print(message)
        print('.'*40 + '?')
        print(self.get_element(i).get_str_japanese())
        k = input('.'*40 + '?\n')
        if k=='q':
            return 'quit'
        elif k=='f':
            self.get_element(i).set_tag('focus')
            print(self.get_element(i).get_str_english())
            self.get_element(i).increment_ctr()
        elif k=='s':
            self.get_element(i).set_tag('skip')
            print(self.get_element(i).get_str_english())
            self.get_element(i).increment_ctr()
        elif k=='':
            print(self.get_element(i).get_str_english())
            self.get_element(i).increment_ctr()
        else:
            self.database[i].ctr = int(k)
        return 'cont'


class Interface:

    def input_sentenses(self):
        sentenses = ''
        lines = sys.stdin.readlines()
        for i in lines:
            sentenses += i
        return sentenses


    def add_new_element(self):
        print('日本語を入力')
        ja = input('> ')
        print('英語を入力')
        en = input('> ')
        self.db.add_new_element(ja,en)

        k = input('続けますか？[y（スペース）/n]')
        if k=='n':
            self.db.save()
        else:
            self.add_new_element()

    def random_quiz_from_english(self):
        size = len(self.db)
        seq = list(range(size))
        random.shuffle(seq)
        try:
            for i in seq:
                c = self.db.quiz_english_japanese(i)
                if c=='quit':
                    self.db.save()
                    return
            self.db.save()
        except KeyboardInterrupt:
            self.db.save()


    def random_quiz_from_japanese(self):
        size = len(self.db)
        seq = list(range(size))
        random.shuffle(seq)
        try:
            for i in seq:
                c = self.db.quiz_japanese_english(i)
                # c = self.db.quiz_english_japanese(i)
                if c=='quit':
                    self.db.save()
                    return
            self.db.save()
        except KeyboardInterrupt:
            self.db.save()


    def __init__(self,inputfile):
        self.db = Database(inputfile)

    def start(self):
        print('** Hello. Welcome to Teppen.')
        k = input('** Enter e [英和] | j [和英] | a [append] | q [quit] > ')
        if k=='q':
            return
        elif k=='e':
            self.random_quiz_from_english()
        elif k=='j':
            self.random_quiz_from_japanese()
        elif k=='a':
            self.add_new_element()



if __name__ == '__main__':
    if (len(sys.argv) != 2):
        # print('Usage: %s filename' % sys.argv[0])
        # quit()
        filename = 'teppen.txt'
    else:
        filename = sys.argv[1]

    face = Interface(filename)
    face.start()




** Hello. Welcome to Teppen.
** Enter e [英和] | j [和英] | a [append] | q [quit] > e

> 1-th [1 counts]... Return | f(ocus) | s(kip) | q(uit)
........................................?
Nana speaks not only English but also French.
........................................?
q


In [0]:
f = open("teppen.txt", "w+")   
a = "ナナは英語だけでなくフランス語も話します。 § Nana speaks not only English but also French. § new § 1"
f.write(a)
f.close()